### Dataset
[https://www.kaggle.com/praveengovi/emotions-dataset-for-nlp](https://www.kaggle.com/praveengovi/emotions-dataset-for-nlp)

predict feeling when provided with a text

### imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM

from numpy import zeros
from keras.layers import Flatten
from keras.layers.core import Activation, Dropout, Dense

### join datasets
> the dataset was already split into train,test,validation...this block joins the datasets into a single dataset so that the dataset split can be carried out later

In [2]:
#column names
labels=["text","feeling"]
# read datasets
chunk1=pd.read_csv("data/test.txt",delimiter=";",names=labels,skiprows=1)
chunk2=pd.read_csv("data/train.txt",delimiter=";",names=labels,skiprows=1)
chunk3=pd.read_csv("data/val.txt",delimiter=";",names=labels,skiprows=1)

# join datasets
data=pd.concat([chunk1,chunk2,chunk3], axis=0)

# reset index since concatinating keeps the orginal index of the datasets
data.reset_index(drop=True, inplace=True)
data.tail()

,text,feeling
19992,im having ssa examination tomorrow in the morn...,sadness
19993,i constantly worry about their fight against n...,joy
19994,i feel its important to share this info for th...,joy
19995,i truly feel that if you are passionate enough...,joy
19996,i feel like i just wanna buy any cute make up ...,joy


### preparation

In [3]:
# cleaning
# all text to lower case
data['text'] = data['text'].apply(lambda x: x.lower())
# removing common words - noise
x_words=["i ","im ","feel ","feeling "]
for x_word in x_words:
    data['text'] = data['text'].apply(lambda x: x.replace(x_word, ' '))
# preview cleaned data
data.head()

,text,feeling
0,updating my blog because shitty,sadness
1,never make her separate from me because don ...,sadness
2,left with my bouquet of red and yellow tulips...,joy
3,was a little vain when did this one,sadness
4,cant walk into a shop anywhere where do not ...,fear


In [4]:
# encoding

tmp_ct=data["feeling"].astype("category")
data["feeling"] = tmp_ct.cat.codes

# used to refer to the category codes
# i.e {0: 'anger', 1: 'fear', 2: 'joy', 3: 'love', 4: 'sadness', 5: 'surprise'}
category_map=dict(enumerate(tmp_ct.cat.categories))

# preview encoded data
data.tail()

,text,feeling
19992,having ssa examination tomorrow in the mornin...,4
19993,constantly worry about their fight against na...,2
19994,its important to share this info for those t...,2
19995,truly that if you are passionate enough abou...,2
19996,like just wanna buy any cute make up see o...,2


### train test validation split

In [5]:
X=data["text"]
y=data["feeling"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [6]:
max_len = 100
max_words = 5000
tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(list(X_train))
vocab_size = len(tokenizer.word_index) + 1

seq= tokenizer.texts_to_sequences(X_train.values)
seq_val = tokenizer.texts_to_sequences(X_val.values)
X_train_seq = pad_sequences(seq,maxlen=max_len)
X_val_seq = pad_sequences(seq_val,maxlen=max_len)

### training

In [7]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100,input_length=100,trainable=False)
model.add(embedding_layer)
model.add(LSTM(100,dropout=0.2))
model.add(Dense(1,activation='softmax'))

# tracking accuracy
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

print(model.summary())
history = model.fit(np.array(X_train_seq), np.array(y_train), epochs=20, verbose=1,batch_size = 128,validation_data=(X_val_seq,y_val))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1398500   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 1,479,001
Trainable params: 80,501
Non-trainable params: 1,398,500
_________________________________________________________________
None
Epoch 1/20
107/107 [==============================] - 30s 262ms/step - loss: 0.0000e+00 - accuracy: 0.1188 - val_loss: 0.0000e+00 - val_accuracy: 0.1191
Epoch 2/20
107/107 [==============================] - 29s 270ms/step - loss: 0.0000e+00 - accuracy: 0.1188 - val_loss: 0.0000e+00 - val_accuracy: 0.1191
Epoch 3/20
107/107 [==============================] - 26s 24